In [ ]:
import os
import zipfile
import json
from google.colab import drive
import PIL
import PIL.Image
import tensorflow as tf
import pathlib

drive.mount('/content/drive')
!pip install unrar
!unrar x /content/drive/MyDrive/data.rar

In [16]:
import keras
import tensorflow as tensorflow
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import PIL
from PIL import Image

TRAINING_DIR = "/content/extracted_images"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "/content/extracted_images"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(45,45),
	class_mode='categorical',
  batch_size=126
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(45,45),
	class_mode='categorical',
  batch_size=126
)

Found 511694 images belonging to 83 classes.
Found 511694 images belonging to 83 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(45, 45, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(82, activation='softmax')
])
model.summary()
model.compile(loss = "categorical_crossentropy", optimizer='rmsprop', metrics=['accuracy'])
history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)
model.save("tapedu.h5")

In [ ]:
import cv2
def prepare(filepath):
    IMG_SIZE = 45
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

prediction = model.predict([prepare('image.jpg')])
print(prediction)